# LAB 3 - Least Squares Solutions for Overdetermined Systems

Chouliaras Andreas 2143

Pappas Apostolos 2109

Gkountouvas Stylianos 


### Overdetermined systems

When we want to solve systems of linear equations, $\hat{y} = X\beta$, we need as many equations as unknowns. When
the number of unknowns is equal to the number of equations there may exist a single \textbf{unique} solution. In the
case where there are more unknowns than equations we say that the system is \textbf{underdetermined} since we
have no way to uniquely solve for every unknown. A more common case is when there are more equations than unknowns, in this case the system is \textbf{overdetermined} and we have many possible solutions, but no single unique one.   

We need to find solutions that are characterized as satisfactory. Thus, the concept of error(also named cost) is introduced. A satisfactory solytion is the one that minimizes the error defined as:

$$\epsilon = X\beta - y \Rightarrow$$

$$\epsilon = \hat{y} - y$$


The following dataset is a good example of what an overdetermined system is.

In [32]:
#install.packages("knitr")
library(knitr)

mls = data.frame( read = c(63,55,60,73,37,68,76,66,63,60,
52,50,36,57,50,42,73,47,68,50),
write = c(57,39,62,67,44,60,63,67,57,46,
49,41,57,52,49,49,62,62,65,52),
science = c(58,53,50,58,39,69,67,61,58,53,
44,44,50,61,47,50,69,53,55,39),
math = c(55,57,67,62,45,64,60,67,54,51,49,
45,42,40,56,43,73,53,62,53),
socst = c(41,46,56,66,46,66,66,66,51,61,
61,56,41,56,46,56,66,61,61,56))

mls

read,write,science,math,socst
63,57,58,55,41
55,39,53,57,46
60,62,50,67,56
73,67,58,62,66
37,44,39,45,46
68,60,69,64,66
76,63,67,60,66
66,67,61,67,66
63,57,58,54,51
60,46,53,51,61


We will try to approach the data above using several techniques. 

### The linear model

The first model that we will use is the simple linear model as presented below:

$$Sci_i = \beta_0 + \beta_1Read_i + \beta_2Math_i + \epsilon_i$$

With R's lm() function we can estimate the above model. We also construct the appropriate $x$ and $y$ matrices that will be used next.

In [33]:
# prepare the data
y =mls$science
x = cbind('(Intercept)'=1,mls[,c('read','math')])
x = as.matrix(x)

In [34]:
# from R
coef(lm(science~read+math,mls))->lma
lma

(Intercept)        read        math 
21.88956117  0.64655253 -0.09174902

Thus, the simple linear model is:

$$Sci_i = 21.89 + 0.65Read_i - 0.091Math_i + \epsilon_i$$

### Direct Matrix Inversion

Another way is to use the so-called normal equations where:

$$X\beta = y \Rightarrow$$

$$[X^TX]\beta = X^Ty \Rightarrow$$

$$\beta = [X^TX]^{-1}X^Ty$$

In [35]:
# the solve() function performs the matrix inversion
solve( t(x) %*%x ) %*% t(x) %*%y ->ols
ols

(Intercept),21.88956117
read,0.64655253
math,-0.09174902


This produces the same estimates which is nice to see. However, inverting a matrix can be diﬃcult, time
consuming, and overall computationally intensive. Below are other methods that break down the solution
into more manageable parts. Speciﬁcally, instead of inverting the full matrix $X^TX$, it can be broken down or
factorized in some way to make the inversion process much less computationally intensive.

### QR method

The QR decomposition, or factorization, takes a matrix A and produces two additional matrices Q and R
that represent an orthogonal matrix and a triangular matrix respectively. The form of the decomposition is

$$A = QR$$

Since $Q$ is orthogonal, $Q^TQ = I$. Also, R is triangular, thus easier to invert.
In the next block of code, we construct the appropriate Q and R matrices.

In [36]:
# QR decomposition to solve
qr.Q(qr(x)) ->Q
qr.R(qr(x)) ->R
# Make sure Q is orthogonal
kable(as.data.frame( t(Q) %*%Q ))



| V1| V2| V3|
|--:|--:|--:|
|  1|  0|  0|
|  0|  1|  0|
|  0|  0|  1|

In [37]:
# What is in R
R

(Intercept),read,math
-4.472136,-256.25339,-245.52026
0.000000,51.24646,30.70651
0.000000,0.00000,-26.01750


Now that we've got $X$ decomposed, we can solve the system as follows:

$$X\beta = y \Rightarrow$$

$$QR\beta = y \Rightarrow$$

$$Q^TQR\beta = Q^Ty \Rightarrow$$

$$R\beta = Q^Ty \Rightarrow$$

$$\beta = R^{-1}Q^Ty$$

That is implemented below. 

In [38]:
# Using QR (Spoiler Alert: this is what the lm() function does on the backend)
solve(R) %*% t(Q) %*%y ->qrs
qrs

(Intercept),21.88956117
read,0.64655253
math,-0.09174902


### Cholesky method

The **Cholesky** decomposition or Cholesky factorization is a decomposition of a Hermitian$(A^TA)$, positive-definite matrix into the product of a lower triangular matrix and its conjugate transpose. The Cholesky decomposition is roughly twice as efficient as the LU decomposition for solving systems of linear equations.

The Cholesky decomposition of a Hermitian positive-definite matrix A is a decomposition of the form $A = R^TR$, where R is a lower triangular matrix with real and positive diagonal entries, and LT denotes the conjugate transpose of L. Every Hermitian positive-definite matrix (and thus also every real-valued symmetric positive-definite matrix) has a unique Cholesky decomposition.

In [39]:
# Cholesky
cv = t(x) %*%x
u = chol(cv)
uinv = solve(u)
uinv %*% t(uinv) %*% t(x) %*%y ->chs
chs

(Intercept),21.88956117
read,0.64655253
math,-0.09174902


### Singular Value Decomposition

This decomposition breaks down our matrix A into three new matrices U, D, and V , such that $A = UDV^T$.
In this case both U and V are orthogonal and D is a diagonal matrix containing the singular values such
that D = $U^TAV$.

The fact that D is diagonal makes this a much easier problem since the inverse of a diagonal matrix is equal
to a matrix with each diagonal element inverted (only for non-zero elements of course!)
As a quick example

In [40]:
A = diag(c(2,4,5))
A

2,0,0
0,4,0
0,0,5


In [41]:
solve(A)

0.5,0.00,0.0
0.0,0.25,0.0
0.0,0.00,0.2


To solve the system, we do:

$$X\beta = y \Rightarrow$$

$$UDV^T\beta = y \Rightarrow$$

$$U^TUDV^T\beta = U^Ty \Rightarrow$$

$$DV^T\beta = U^Ty \Rightarrow$$

$$D^{-1}DV^T\beta = D^{-1}U^Ty \Rightarrow$$

$$V^T\beta = D^{-1}U^Ty \Rightarrow$$

$$VV^T\beta = VD^{-1}U^Ty \Rightarrow$$

$$\beta = VD^{-1}U^Ty$$

What follows is the implementation of the algorithnm above.

In [42]:
# Singular Value Decomposition
dcomp = svd(x)
V =dcomp$v
# orthogonal eigenvectors
D = diag(dcomp$d) # diagonal singular values
U =dcomp$u
# orthogonal
V %*% solve(D) %*% t(U) %*%y ->svs
svs

21.88956117
0.64655253
-0.09174902


### How do they compare?

In [43]:
results = data.frame('R'=lma, 'OLS'=ols, 'QR'=qrs, 'Cholesky'=chs, 'SVD'=svs)
kable(results,digits=5)



|            |        R|      OLS|       QR| Cholesky|      SVD|
|:-----------|--------:|--------:|--------:|--------:|--------:|
|(Intercept) | 21.88956| 21.88956| 21.88956| 21.88956| 21.88956|
|read        |  0.64655|  0.64655|  0.64655|  0.64655|  0.64655|
|math        | -0.09175| -0.09175| -0.09175| -0.09175| -0.09175|

We see that the estimates are exactly the same. It is somehow expected since we actually solved the same problem/system, using several different approaches.

### Interaction eﬀect estimates?

Here we include an interaction eﬀect between math and read, thus our model becomes,

$$Sci_i = \beta_0 + \beta_1Read_i + \beta_2Math_i + \beta_3(Math_i\times Read_i) + \epsilon_i$$

In [44]:
# interactions?
x = cbind(x,'intrxn'=mls$read*mls$math)
x = as.matrix(x)
# from R
lm0 = lm(science~read*math,mls)
lm0$coef ->beta1
# Traditional Least squares
solve( t(x) %*%x ) %*% t(x) %*%y ->beta2
# QR decomposition to solve
qr.Q(qr(x)) ->Q
qr.R(qr(x)) ->R
solve(R) %*% t(Q) %*%y ->beta3
# Cholesky
cv = t(x) %*%x
u = chol(cv)
uinv = solve(u)
uinv %*% t(uinv) %*% t(x) %*%y ->beta4
# Singular Value Decomposition
dcomp = svd(x)
V =dcomp$v
# orthogonal eigenvectors
D = diag(dcomp$d) # diagonal singular values
U =dcomp$u
# orthogonal
V %*% solve(D) %*% t(U) %*%y ->beta5

### How do they compare?

In [45]:
res = data.frame('R'=beta1, 'OLS'=beta2, 'QR'=beta3, 'Cholesky'=beta4, 'SVD'=beta5)
kable(res,digits=5)



|            |         R|       OLS|        QR|  Cholesky|       SVD|
|:-----------|---------:|---------:|---------:|---------:|---------:|
|(Intercept) | 107.29380| 107.29380| 107.29380| 107.29380| 107.29380|
|read        |  -0.78323|  -0.78323|  -0.78323|  -0.78323|  -0.78323|
|math        |  -1.78291|  -1.78291|  -1.78291|  -1.78291|  -1.78291|
|read:math   |   0.02772|   0.02772|   0.02772|   0.02772|   0.02772|

### Standard errors of estimates

We can also use aspects of these matrix methods for model assessment as well. In
particular we may be interested in computing \textbf{standard errors} of the parameters as well as of the estimates.
Looking for standard errors of estimates for QR decomposition it will be helpful to know that errors are
deﬁned as:

$$\epsilon = (y - QQ^Ty),$$

$$MSE = \frac{1}{n-p}\epsilon^T\epsilon,$$

$$SE_{\beta} = \sqrt{\frac{MSE}{SS_x}},$$

which in the case of QR factorization becomes,

$$SE_{\beta} = \sqrt{\frac{MSE}{R^TR}}$$




In [46]:
n = nrow(x)
p = ncol(x)
# e = (I - QQ')y
( diag(20) -Q %*% t(Q) ) %*%y ->err3
# MSE = 1/n-p (e'e)
MSE =1/(n-p) * t(err3) %*%err3
# SEb = (MSE/SSx)^.5
SEb = sqrt( MSE * diag( solve( t(R) %*%R ) ) )
kable(data.frame('Est' =beta3,'Std.Err'=SEb, 't val'=beta3/SEb),digits=c(5,5,3))

Warning message in MSE * diag(solve(t(R) %*% R)):
"Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
"



|            |       Est|  Std.Err|  t.val|
|:-----------|---------:|--------:|------:|
|(Intercept) | 107.29380| 44.13585|  2.431|
|read        |  -0.78323|  0.74634| -1.049|
|math        |  -1.78291|  0.88619| -2.012|
|intrxn      |   0.02772|  0.01411|  1.964|

In [47]:
sqrt(MSE)

5.455545


In [48]:
summary(lm0)


Call:
lm(formula = science ~ read * math, data = mls)

Residuals:
   Min     1Q Median     3Q    Max 
-8.084 -5.258  1.223  2.603  8.454 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept) 107.29380   44.13585   2.431   0.0272 *
read         -0.78323    0.74634  -1.049   0.3096  
math         -1.78291    0.88619  -2.012   0.0614 .
read:math     0.02772    0.01411   1.964   0.0671 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 5.456 on 16 degrees of freedom
Multiple R-squared:  0.6858,	Adjusted R-squared:  0.6269 
F-statistic: 11.64 on 3 and 16 DF,  p-value: 0.0002689
